In [1]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from sklearn.model_selection import train_test_split

C:\Users\pavan\AppData\Local\Temp\ipykernel_12560\3984744096.py:4: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [3]:
data_dir = 'data'

In [4]:
image_exts = ['jpeg','jpg','png','bmp']

In [5]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
        image_path = os.path.join(data_dir,image_class,image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('image doesnt exist {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('issue with the image {}'.format(image_path))

In [6]:
images = []
labels = []
input_size = (256,256)
batch_size = 32

In [27]:
data_dir = 'data'
classes = ['8x','dove','head and shoulders','Loreal paris','mamaearth','p&g','saslic','wow skincare']
num_classes = len(classes)
print(num_classes)

8


In [28]:
def load_data(data_dir):

    images = []
    labels = []

    for label, brand_name in enumerate(os.listdir(data_dir)):
        brand_dir = os.path.join(data_dir, brand_name)
        for image_name in os.listdir(brand_dir):
            image_path = os.path.join(brand_dir, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (256, 256))
            images.append(image)
            labels.append(label)
    images = np.array(images, dtype="float32") / 255.0
    labels = np.array(labels)
    return images, labels


images, labels = load_data(data_dir)
print("Data shape:", images.shape)
print("Labels shape:", labels.shape)

Data shape: (1244, 256, 256, 3)
Labels shape: (1244,)


In [29]:
X_train, X_test, y_train, y_test = train_test_split(images, labels,train_size=0.8, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [30]:
model1 = Sequential()

model1.add(Conv2D(32,(3,3), activation='relu',padding='same' , input_shape=(256,256,3)))
model1.add(MaxPooling2D((2,2)))

model1.add(Conv2D(64,(3,3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2)))

model1.add(Conv2D(128,(3,3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2)))

model1.add(Conv2D(128,(3,3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2)))

model1.add(Conv2D(512,(3,3), activation='relu', padding='same'))
model1.add(MaxPooling2D((2, 2)))

model1.add(Flatten())
model1.add(Dense(512,activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(8,activation='softmax'))

In [31]:
# Compile the model
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [32]:
# Print the model summary
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 32, 32, 128)     

In [33]:
model1.fit(X_train, y_train, batch_size=32, epochs=12, validation_data=(X_val, y_val))

Epoch 1/12
24/24 [==============================] - 32s 1s/step - loss: 2.2975 - accuracy: 0.1180 - val_loss: 2.0783 - val_accuracy: 0.2129
Epoch 2/12
24/24 [==============================] - 31s 1s/step - loss: 2.0132 - accuracy: 0.1997 - val_loss: 1.8286 - val_accuracy: 0.2851
Epoch 3/12
24/24 [==============================] - 31s 1s/step - loss: 1.7121 - accuracy: 0.3861 - val_loss: 1.5921 - val_accuracy: 0.4980
Epoch 4/12
24/24 [==============================] - 32s 1s/step - loss: 1.3906 - accuracy: 0.5362 - val_loss: 1.5058 - val_accuracy: 0.5301
Epoch 5/12
24/24 [==============================] - 32s 1s/step - loss: 1.1615 - accuracy: 0.5697 - val_loss: 1.4000 - val_accuracy: 0.5944
Epoch 6/12
24/24 [==============================] - 33s 1s/step - loss: 1.0004 - accuracy: 0.6783 - val_loss: 1.4718 - val_accuracy: 0.5863
Epoch 7/12
24/24 [==============================] - 32s 1s/step - loss: 0.8090 - accuracy: 0.7279 - val_loss: 1.3525 - val_accuracy: 0.6024
Epoch 8/12
24/24 [==

In [34]:
test_loss, test_acc = model1.evaluate(X_test, y_test)

8/8 [==============================] - 2s 286ms/step - loss: 1.2413 - accuracy: 0.7309


In [35]:
print(test_acc)

0.7309237122535706


In [36]:
model1.save('model.h5')